In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import sys

# 1️⃣ Chuyển working directory tới root project (nơi có thư mục src)
%cd /content/drive/MyDrive/chatbot

# 2️⃣ Thêm folder root vào sys.path để Python tìm src
sys.path.append(os.getcwd())

# 3️⃣ Kiểm tra
!ls
# Nên thấy thư mục src, app_gradio.py, notebooks, v.v.


/content/drive/MyDrive/chatbot
app_gradio.ipynb  data		    make_data.ipynb  src
app_gradio.py	  evaluation.ipynb  models


In [ ]:
import json

file_path = "data/rag_questions_context.jsonl"

fields = set()

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        fields.update(item.keys())
        break   # chỉ cần đọc dòng đầu tiên để xem structure

print("Các trường có trong file JSONL:")
for k in fields:
    print("-", k)


Các trường có trong file JSONL:
- num_clauses
- combined_text
- law_title
- question
- sub_category
- article_link
- retrieved_doc
- article_title
- article_id


In [ ]:
import json
import re

input_path = "data/rag_questions_context.jsonl"
output_path = "data/eval.jsonl"

def clean_combined_text(text):
    """
    1. Xóa 'Khoản X:' ở đầu
    2. Xóa dạng 'Khoản X:' ở bất kỳ dòng nào
    """
    # Xóa "Khoản X:" ở đầu chuỗi
    text = re.sub(r"^Khoản\s*\d+\s*:\s*", "", text.strip(), flags=re.IGNORECASE)

    # Xóa "Khoản X:" ở giữa nội dung
    text = re.sub(r"Khoản\s*\d+\s*:\s*", "", text, flags=re.IGNORECASE)

    return text.strip()


def extract_article_id(text):
    """
    Tạo article_id từ cụm dạng:
    'luat_lao_dong dieu_1 khoan_0'
    và chuyển spaces thành dấu gạch dưới.
    """
    # Tìm cụm như: luat_lao_dong dieu_1 khoan_0
    match = re.search(
        r"([a-zA-Z_]+?\s+dieu_\d+\s+khoan_\d+)",
        text
    )
    if match:
        # Thay space thành underscore
        return match.group(1).strip().replace(" ", "_")

    # fallback: lấy 1 cụm gần nhất giống logic luật
    match2 = re.search(r"[a-zA-Z_]+?\s+dieu_\d+", text)
    if match2:
        return match2.group(0).strip().replace(" ", "_")

    return "unknown_id"


# ===========================
# 🔥 READ + PROCESS + WRITE
# ===========================

with open(input_path, "r", encoding="utf-8") as fin, \
     open(output_path, "w", encoding="utf-8") as fout:

    for line in fin:
        item = json.loads(line)

        raw_text = item.get("combined_text", "")
        question = item.get("question", "")

        # Step 1: clean text
        cleaned_text = clean_combined_text(raw_text)

        # Step 2: generate new article_id
        new_id = extract_article_id(raw_text)

        # Ghi file mới
        new_item = {
            "id": new_id,
            "question": question,
            "combined_text": cleaned_text
        }

        fout.write(json.dumps(new_item, ensure_ascii=False) + "\n")

print("✅ Done! File saved to:", output_path)


✅ Done! File saved to: data/eval.jsonl


In [4]:
!pip install rank-bm25 unidecode pinecone python-dotenv sentence-transformers -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.5 MB/s eta 0:00:00


In [ ]:
import json
from tqdm import tqdm
from src.bm25_manager import BM25Retriever

# ===========================================================
# Load BM25 retriever
# ===========================================================
bm25_path = "data/keywords_db.jsonl"
retriever = BM25Retriever(jsonl_path=bm25_path)

# ===========================================================
# Load evaluation dataset
# ===========================================================
eval_path = "data/eval.jsonl"
eval_data = []

with open(eval_path, "r", encoding="utf-8") as f:
    for line in f:
        eval_data.append(json.loads(line))

print(f"Loaded {len(eval_data)} evaluation samples")

# ===========================================================
# Metrics container
# ===========================================================
def evaluate_bm25(eval_data, retriever, ks=[1, 3, 5, 10]):
    results = {k: {"hit": 0, "mrr": 0.0} for k in ks}
    total = len(eval_data)

    for item in tqdm(eval_data):
        query = item["question"]
        true_id = item["id"]

        # BM25 search
        docs = retriever.search(query, top_k=max(ks))

        # lấy list id BM25
        retrieved_ids = [d["id"] for d in docs]

        # vị trí của ground truth
        rank = None
        for i, rid in enumerate(retrieved_ids):
            if rid == true_id:
                rank = i + 1   # rank bắt đầu từ 1
                break

        # cập nhật metric
        for k in ks:
            # Hit@K / Recall@K
            if rank is not None and rank <= k:
                results[k]["hit"] += 1

            # MRR@K
            if rank is not None and rank <= k:
                results[k]["mrr"] += 1.0 / rank

    # normalize
    for k in ks:
        results[k]["recall"] = results[k]["hit"] / total
        results[k]["hit_rate"] = results[k]["hit"] / total
        results[k]["mrr"] /= total

    return results


# ===========================================================
# Run evaluation
# ===========================================================
metrics = evaluate_bm25(eval_data, retriever)

# ===========================================================
# Print results
# ===========================================================
print("\n===== 📊 BM25 RAG Evaluation Results =====")
for k, v in metrics.items():
    print(f"\nTop-{k}:")
    print(f"  Recall@{k}:   {v['recall']:.4f}")
    print(f"  Hit@{k}:      {v['hit_rate']:.4f}")
    print(f"  MRR@{k}:      {v['mrr']:.4f}")


[BM25] Loaded 14000 documents from /content/drive/MyDrive/chatbot/data/keywords_db.jsonl
Loaded 8152 evaluation samples


100%|██████████| 8152/8152 [21:29<00:00,  6.32it/s]


===== 📊 BM25 RAG Evaluation Results =====

Top-1:
  Recall@1:   0.0601
  Hit@1:      0.0601
  MRR@1:      0.0601

Top-3:
  Recall@3:   0.0952
  Hit@3:      0.0952
  MRR@3:      0.0750

Top-5:
  Recall@5:   0.1300
  Hit@5:      0.1300
  MRR@5:      0.0828

Top-10:
  Recall@10:   0.1998
  Hit@10:      0.1998
  MRR@10:      0.0920


In [ ]:
import json
from tqdm import tqdm
from src.pinecone_manager import search_pinecone  # import hàm Pinecone search

# ===========================================================
# Load evaluation dataset
# ===========================================================
eval_path = "data/eval.jsonl"
eval_data = []

with open(eval_path, "r", encoding="utf-8") as f:
    for line in f:
        eval_data.append(json.loads(line))

print(f"Loaded {len(eval_data)} evaluation samples")

# ===========================================================
# Metrics container
# ===========================================================
def evaluate_rag(eval_data, top_ks=[1, 3, 5, 10]):
    results = {k: {"hit": 0, "mrr": 0.0} for k in top_ks}
    total = len(eval_data)

    for item in tqdm(eval_data, desc="Evaluating RAG"):
        query = item["question"]
        true_id = item["id"]

        # Pinecone search
        docs = search_pinecone(query, top_k=max(top_ks))
        retrieved_ids = [d["id"] for d in docs]

        # find rank of ground truth
        rank = None
        for i, rid in enumerate(retrieved_ids):
            if rid == true_id:
                rank = i + 1
                break

        # update metrics
        for k in top_ks:
            if rank is not None and rank <= k:
                results[k]["hit"] += 1
                results[k]["mrr"] += 1.0 / rank

    # normalize
    for k in top_ks:
        results[k]["recall"] = results[k]["hit"] / total
        results[k]["hit_rate"] = results[k]["hit"] / total
        results[k]["mrr"] /= total

    return results

# ===========================================================
# Run evaluation
# ===========================================================
metrics = evaluate_rag(eval_data)

# ===========================================================
# Print results
# ===========================================================
print("\n===== 📊 Pinecone RAG Evaluation Results =====")
for k, v in metrics.items():
    print(f"\nTop-{k}:")
    print(f"  Recall@{k}:   {v['recall']:.4f}")
    print(f"  Hit@{k}:      {v['hit_rate']:.4f}")
    print(f"  MRR@{k}:      {v['mrr']:.4f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Loaded 8152 evaluation samples


Evaluating RAG: 100%|██████████| 8152/8152 [14:18<00:00,  9.49it/s]


===== 📊 Pinecone RAG Evaluation Results =====

Top-1:
  Recall@1:   0.1043
  Hit@1:      0.1043
  MRR@1:      0.1043

Top-3:
  Recall@3:   0.2732
  Hit@3:      0.2732
  MRR@3:      0.1769

Top-5:
  Recall@5:   0.3577
  Hit@5:      0.3577
  MRR@5:      0.1962

Top-10:
  Recall@10:   0.4677
  Hit@10:      0.4677
  MRR@10:      0.2110


In [ ]:
import json
from tqdm import tqdm
from src.bm25_manager import BM25Retriever
from src.ensemble_retriever import build_context  # import hàm build_context mới có pinecone_weight

# ===========================================================
# Load evaluation dataset
# ===========================================================
eval_path = "data/eval.jsonl"
eval_data = [json.loads(l) for l in open(eval_path, "r", encoding="utf-8")]
print(f"Loaded {len(eval_data)} evaluation samples")

# ===========================================================
# Initialize BM25 retriever + cache
# ===========================================================
bm25_retriever = BM25Retriever(jsonl_path="data/keywords_db.jsonl")
retrieval_cache = []

# ===========================================================
# Evaluation metrics function
# ===========================================================
def evaluate_rag(eval_data, bm25_retriever, retrieval_cache=None, top_ks=[1,3,5,10], pinecone_weight=1.5):
    if retrieval_cache is None:
        retrieval_cache = []

    results = {k: {"hit":0, "mrr":0.0} for k in top_ks}
    total = len(eval_data)

    for item in tqdm(eval_data, desc="Evaluating RAG"):
        query = item["question"]
        true_id = item["id"]

        # use build_context with pinecone_weight
        context_text, final_docs, retrieval_cache, cache_hit, cosine_score = build_context(
            query,
            retrieval_cache,
            bm25_retriever=bm25_retriever,
            pinecone_weight=pinecone_weight
        )

        retrieved_ids = [d["id"] for d in final_docs]

        # find rank of ground truth
        rank = None
        for i, rid in enumerate(retrieved_ids):
            if rid == true_id:
                rank = i + 1
                break

        # update metrics
        for k in top_ks:
            if rank is not None and rank <= k:
                results[k]["hit"] += 1
                results[k]["mrr"] += 1.0 / rank

    # normalize
    for k in top_ks:
        results[k]["recall"] = results[k]["hit"]/total
        results[k]["hit_rate"] = results[k]["hit"]/total
        results[k]["mrr"] /= total

    return results, retrieval_cache

# ===========================================================
# Run evaluation
# ===========================================================
metrics, retrieval_cache = evaluate_rag(
    eval_data,
    bm25_retriever=bm25_retriever,
    retrieval_cache=retrieval_cache,
    pinecone_weight=1.5
)

# ===========================================================
# Print results
# ===========================================================
print("\n===== 📊 RAG Evaluation Results (Pinecone weight=1.5) =====")
for k, v in metrics.items():
    print(f"Top-{k}: Recall@{k}: {v['recall']:.4f}, Hit@{k}: {v['hit_rate']:.4f}, MRR@{k}: {v['mrr']:.4f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loaded 8152 evaluation samples
[BM25] Loaded 14000 documents from /content/drive/MyDrive/chatbot/data/keywords_db.jsonl


Evaluating RAG: 100%|██████████| 8152/8152 [28:46<00:00,  4.72it/s]


===== 📊 RAG Evaluation Results (Pinecone weight=1.5) =====
Top-1: Recall@1: 0.0812, Hit@1: 0.0812, MRR@1: 0.0812
Top-3: Recall@3: 0.2041, Hit@3: 0.2041, MRR@3: 0.1319
Top-5: Recall@5: 0.3180, Hit@5: 0.3180, MRR@5: 0.1577
Top-10: Recall@10: 0.4679, Hit@10: 0.4679, MRR@10: 0.1783


In [ ]:
import json
import pandas as pd
from tqdm import tqdm
from src.bm25_manager import BM25Retriever
from src.ensemble_retriever import build_context

# ===========================================================
# Load evaluation dataset
# ===========================================================
eval_path = "data/eval.jsonl"
eval_data = [json.loads(l) for l in open(eval_path, "r", encoding="utf-8")]
print(f"Loaded {len(eval_data)} evaluation samples")

# ===========================================================
# Initialize BM25 retriever
# ===========================================================
bm25_retriever = BM25Retriever(jsonl_path="data/keywords_db.jsonl")


# ===========================================================
# Evaluation metrics function
# ===========================================================
def evaluate_rag(eval_data, bm25_retriever, retrieval_cache=None, top_ks=[1,3,5,10], pinecone_weight=1.5):
    if retrieval_cache is None:
        retrieval_cache = []

    results = {k: {"hit":0, "mrr":0.0} for k in top_ks}
    total = len(eval_data)

    for item in tqdm(eval_data, desc=f"Evaluating RAG (weight={pinecone_weight})"):
        query = item["question"]
        true_id = item["id"]

        context_text, final_docs, retrieval_cache, cache_hit, cosine_score = build_context(
            query,
            retrieval_cache,
            bm25_retriever=bm25_retriever,
            pinecone_weight=pinecone_weight
        )

        retrieved_ids = [d["id"] for d in final_docs]

        # find rank of ground truth
        rank = None
        for i, rid in enumerate(retrieved_ids):
            if rid == true_id:
                rank = i + 1
                break

        # update metrics
        for k in top_ks:
            if rank is not None and rank <= k:
                results[k]["hit"] += 1
                results[k]["mrr"] += 1.0 / rank

    # normalize
    for k in top_ks:
        results[k]["recall"] = results[k]["hit"]/total
        results[k]["hit_rate"] = results[k]["hit"]/total
        results[k]["mrr"] /= total

    return results


# ===========================================================
# Run evaluation for multiple pinecone weights
# ===========================================================
pinecone_weights = [0.5, 1.5, 2, 3, 5]
top_ks = [1,3,5,10]

rows = []   # for summary table

for weight in pinecone_weights:
    retrieval_cache = []  # reset cache

    metrics = evaluate_rag(
        eval_data,
        bm25_retriever=bm25_retriever,
        retrieval_cache=retrieval_cache,
        pinecone_weight=weight
    )

    # ---- print result immediately ----
    print(f"\n===== 📊 RAG Evaluation Results (Pinecone weight={weight}) =====")
    for k, v in metrics.items():
        print(f"Top-{k}: Recall@{k}: {v['recall']:.4f}, Hit@{k}: {v['hit_rate']:.4f}, MRR@{k}: {v['mrr']:.4f}")

    # save to table
    for k in top_ks:
        rows.append({
            "Pinecone_weight": weight,
            "Top-K": k,
            "Recall": metrics[k]["recall"],
            "Hit_rate": metrics[k]["hit_rate"],
            "MRR": metrics[k]["mrr"]
        })


# ===========================================================
# Final summary table
# ===========================================================
df = pd.DataFrame(rows)

print("\n\n==================== 📌 TỔNG KẾT SO SÁNH TẤT CẢ WEIGHTS ====================")
print(df.pivot(index="Top-K", columns="Pinecone_weight", values=["Recall","Hit_rate","MRR"]))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

Evaluating RAG (weight=1.5): 100%|██████████| 8152/8152 [28:12<00:00,  4.82it/s]



===== 📊 RAG Evaluation Results (Pinecone weight=1.5) =====
Top-1: Recall@1: 0.0812, Hit@1: 0.0812, MRR@1: 0.0812
Top-3: Recall@3: 0.2041, Hit@3: 0.2041, MRR@3: 0.1319
Top-5: Recall@5: 0.3180, Hit@5: 0.3180, MRR@5: 0.1577
Top-10: Recall@10: 0.4679, Hit@10: 0.4679, MRR@10: 0.1783


Evaluating RAG (weight=2): 100%|██████████| 8152/8152 [27:34<00:00,  4.93it/s]



===== 📊 RAG Evaluation Results (Pinecone weight=2) =====
Top-1: Recall@1: 0.0855, Hit@1: 0.0855, MRR@1: 0.0855
Top-3: Recall@3: 0.2110, Hit@3: 0.2110, MRR@3: 0.1371
Top-5: Recall@5: 0.3189, Hit@5: 0.3189, MRR@5: 0.1615
Top-10: Recall@10: 0.4679, Hit@10: 0.4679, MRR@10: 0.1820


Evaluating RAG (weight=3):  75%|███████▍  | 6109/8152 [20:49<06:03,  5.62it/s]

In [5]:
import json
import pandas as pd
from tqdm import tqdm
from src.bm25_manager import BM25Retriever
from src.ensemble_retriever import build_context

# ===========================================================
# Load evaluation dataset
# ===========================================================
eval_path = "data/eval.jsonl"
eval_data = [json.loads(l) for l in open(eval_path, "r", encoding="utf-8")]
print(f"Loaded {len(eval_data)} evaluation samples")

# ===========================================================
# Initialize BM25 retriever
# ===========================================================
bm25_retriever = BM25Retriever(jsonl_path="data/keywords_db.jsonl")


# ===========================================================
# Evaluation metrics function
# ===========================================================
def evaluate_rag(eval_data, bm25_retriever, retrieval_cache=None, top_ks=[1,3,5,10], pinecone_weight=1.5):
    if retrieval_cache is None:
        retrieval_cache = []

    results = {k: {"hit":0, "mrr":0.0} for k in top_ks}
    total = len(eval_data)

    for item in tqdm(eval_data, desc=f"Evaluating RAG (weight={pinecone_weight})"):
        query = item["question"]
        true_id = item["id"]

        context_text, final_docs, retrieval_cache, cache_hit, cosine_score = build_context(
            query,
            retrieval_cache,
            bm25_retriever=bm25_retriever,
            pinecone_weight=pinecone_weight
        )

        retrieved_ids = [d["id"] for d in final_docs]

        # find rank of ground truth
        rank = None
        for i, rid in enumerate(retrieved_ids):
            if rid == true_id:
                rank = i + 1
                break

        # update metrics
        for k in top_ks:
            if rank is not None and rank <= k:
                results[k]["hit"] += 1
                results[k]["mrr"] += 1.0 / rank

    # normalize
    for k in top_ks:
        results[k]["recall"] = results[k]["hit"]/total
        results[k]["hit_rate"] = results[k]["hit"]/total
        results[k]["mrr"] /= total

    return results


# ===========================================================
# Run evaluation for multiple pinecone weights
# ===========================================================
pinecone_weights = [3, 5]
top_ks = [1,3,5,10]

rows = []   # for summary table

for weight in pinecone_weights:
    retrieval_cache = []  # reset cache

    metrics = evaluate_rag(
        eval_data,
        bm25_retriever=bm25_retriever,
        retrieval_cache=retrieval_cache,
        pinecone_weight=weight
    )

    # ---- print result immediately ----
    print(f"\n===== 📊 RAG Evaluation Results (Pinecone weight={weight}) =====")
    for k, v in metrics.items():
        print(f"Top-{k}: Recall@{k}: {v['recall']:.4f}, Hit@{k}: {v['hit_rate']:.4f}, MRR@{k}: {v['mrr']:.4f}")

    # save to table
    for k in top_ks:
        rows.append({
            "Pinecone_weight": weight,
            "Top-K": k,
            "Recall": metrics[k]["recall"],
            "Hit_rate": metrics[k]["hit_rate"],
            "MRR": metrics[k]["mrr"]
        })


# ===========================================================
# Final summary table
# ===========================================================
df = pd.DataFrame(rows)

print("\n\n==================== 📌 TỔNG KẾT SO SÁNH TẤT CẢ WEIGHTS ====================")
print(df.pivot(index="Top-K", columns="Pinecone_weight", values=["Recall","Hit_rate","MRR"]))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

Loaded 8152 evaluation samples
[BM25] Loaded 14000 documents from /content/drive/MyDrive/chatbot/data/keywords_db.jsonl


Evaluating RAG (weight=3): 100%|██████████| 8152/8152 [23:48<00:00,  5.71it/s]



===== 📊 RAG Evaluation Results (Pinecone weight=3) =====
Top-1: Recall@1: 0.0883, Hit@1: 0.0883, MRR@1: 0.0883
Top-3: Recall@3: 0.2142, Hit@3: 0.2142, MRR@3: 0.1406
Top-5: Recall@5: 0.3192, Hit@5: 0.3192, MRR@5: 0.1644
Top-10: Recall@10: 0.4677, Hit@10: 0.4677, MRR@10: 0.1848


Evaluating RAG (weight=5): 100%|██████████| 8152/8152 [23:24<00:00,  5.80it/s]


===== 📊 RAG Evaluation Results (Pinecone weight=5) =====
Top-1: Recall@1: 0.0915, Hit@1: 0.0915, MRR@1: 0.0915
Top-3: Recall@3: 0.2172, Hit@3: 0.2172, MRR@3: 0.1441
Top-5: Recall@5: 0.3199, Hit@5: 0.3199, MRR@5: 0.1673
Top-10: Recall@10: 0.4677, Hit@10: 0.4677, MRR@10: 0.1877


==================== 📌 TỔNG KẾT SO SÁNH TẤT CẢ WEIGHTS ====================
                   Recall            Hit_rate                 MRR          
Pinecone_weight         3         5         3         5         3         5
Top-K                                                                      
1                0.088322  0.091511  0.088322  0.091511  0.088322  0.091511
3                0.214181  0.217247  0.214181  0.217247  0.140559  0.144075
5                0.319185  0.319921  0.319185  0.319921  0.164350  0.167272
10               0.467738  0.467738  0.467738  0.467738  0.184834  0.187654
